# NDWI of Indonesia Floods

In [1]:
# the regulars
import geopandas as gpd
import matplotlib.pyplot as plt

# work with dates
from datetime import datetime as dt

# allow images to display
from IPython.display import Image

# earth engine
import ee

/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.1-CAPI-1.13.3) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 


Exception: Problem requesting tokens. Please try again.  HTTP Error 400: Bad Request b'{\n  "error": "invalid_request",\n  "error_description": "Missing required parameter: code"\n}'

## Set some parameters

In [4]:
# coordinates of the 2020 Kyushu Region Floods
lat = -6.2088
lon = 106.8456

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2019-01-01'

# end date
end_date = '2019-12-31'

In [5]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(5000) # meters

## Sentinel-2 data

In [6]:
# get the satellite data
# landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
#             .filterBounds(roi)\
#             .filterDate(start_date,end_date)\
#             .filter(ee.Filter.lt('CLOUD_COVER', 20))

In [7]:
# get the satellite data (Sentinel)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterBounds(roi)\
            .filterDate(start_date, end_date)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))

In [8]:
# how many images did we get?
print('Total number:', sentinel.size().getInfo())

Total number: 7


In [9]:
# put the images in a list
sentinel_list = sentinel.toList(sentinel.size());

In [10]:
# set some parameters for image
parameters = {
                'min': 0,
                'max': 2500,
                'dimensions': 800,
                'bands': ['B4', 'B3', 'B2'],
                'region':roi
             }

In [11]:
# loop through each image and display it
for i in range(sentinel.size().getInfo()):

    # when was this image taken?
    date = ee.Date(ee.Image(sentinel_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    
    # cloud cover
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)))
    

Image # 0 2019-04-25 03:19:46 Cloud cover: 2.125039


Image # 1 2019-05-05 03:19:44 Cloud cover: 4.346857


Image # 2 2019-06-09 03:19:47 Cloud cover: 4.371452


Image # 3 2019-06-24 03:19:44 Cloud cover: 3.473744


Image # 4 2019-06-29 03:19:52 Cloud cover: 0.926563


Image # 5 2019-06-29 03:19:48 Cloud cover: 0.030182


Image # 6 2019-07-14 03:19:45 Cloud cover: 0.75867


## Normalized Difference Water Index (NDWI)

NDWI = (green – NIR) / (green + NIR)

For Sentinel data, green = B3 and the NIR is B8

In [12]:
# ndwi palette: red is low water, blue is high water
palette = ['red', 'yellow', 'green', 'blue'];

# notice, no bands here because we will calculate ndwi per image
ndwi_parameters = {'min': -1,
                   'max': 0.5,
                   'dimensions': 800,
                   'palette': palette,
                   'region': roi}

In [13]:
# loop through each image and display it
for i in range(sentinel.size().getInfo()):

    # first, calculate ndwi for each image
    ndwi = ee.Image(sentinel_list.get(i)).normalizedDifference(['B3', 'B8'])
    
    # when was this image taken?
    date = ee.Date(ee.Image(sentinel_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ndwi.getThumbUrl(ndwi_parameters)))

Image # 0 2019-04-25 03:19:46


Image # 1 2019-05-05 03:19:44


Image # 2 2019-06-09 03:19:47


Image # 3 2019-06-24 03:19:44


Image # 4 2019-06-29 03:19:52


Image # 5 2019-06-29 03:19:48


Image # 6 2019-07-14 03:19:45


In [14]:
import folium
from folium import plugins

In [15]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [16]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [17]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)


# loop through each image and display it
for i in range(sentinel.size().getInfo()):

    # first, calculate ndwi for each image
    ndwi = ee.Image(sentinel_list.get(i)).normalizedDifference(['B3', 'B8'])
    
    # when was this image taken?
    date = ee.Date(ee.Image(sentinel_list.get(i)).get('system:time_start'))
    time = date.getInfo()['value']/1000
    date = dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
        
    my_map.add_ee_layer(ndwi, 
                        ndwi_parameters, 
                        name=date)


# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

In [18]:
my_map.save('kumamoto.html')